## Ссылки
1. [Напряжённость гравитационного поля (Вики)](https://ru.wikipedia.org/wiki/Напряжённость_гравитационного_поля)

In [ ]:

%pip install matplotlib
import matplotlib

In [ ]:
# Напряжённость гравитационного поля (через силу всемирного тяготения)
# m = [Кг] - Масса объекта
# r = [м]  - Расстоание от точки до центра массы земли
def e(m: float, r: float) -> float:
    # Гравитационная постоянная
    g = 6.67428e-11
    # Масса земли (кг)
    earth = 5.9742e+24
    # Сила тяготения между землёй и 
    f = g * (m * earth) / r**2
    return f/m